**Author**: gp1981

**Purpose**: Calculate Earnings Yield (EY) and Return on Capital (ROC) for stocks and rank them as detailed in the "The little book that still beats the market" of J. Greenblatt.

***Disclaimer***: This code is provided as-is without any warranty.

Basic instructions to run the code:

1.   Create in VS code virtual environment venv

2.  /.venv/Scripts/activate.bat (for Mac OS)

3.  python -m pip install python-dotenv (for API Key management). Similarly for pandas, matplotlib

4.  create a file .env (starting with dot .) in the root of your project folder

5.  open the .env file in VS code and include API_KEY=your API Key from financialmodellingprep.com


---------------------------------------------------------

In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
import matplotlib as plt


# Load the API key from .env
load_dotenv()
API_KEY = os.environ.get('API_KEY')


# Load repository functions
from Functions.analysis import calculate_ey, calculate_roc, rank_stocks
from Functions.api import *

------------------------------------------------------------------------

The following functions to fetch the stock list and financial data based on financialmodellingprep API via the functions:

-   `fetch_stock_list()`

-   `fetch_financial_statement_symbol_list()`

The symbol list from Financialmodelingprep is downloaded and merged with the list of symbols that have financial reports (e.g. stock of a company)

Finally we filter the symbols of companies that are not traded in US.
------------------------------------------------------------------------



In [3]:
# Fetch stock list and financial statement symbol list
stock_list_data = fetch_stock_list(API_KEY)
symbol_list_data = fetch_financial_statement_symbol_list(API_KEY)

# Rename the column in symbol_list_data
symbol_list_data.rename(columns={0: "symbol"}, inplace=True)

# Merge stock data with financial statement symbol list
merged_data = pd.merge(stock_list_data, symbol_list_data, on="symbol")

# Filter stocks based on exchangeShortName
filtered_data = merged_data[merged_data["exchangeShortName"].isin(["ASX", "ETF", "NASDAQ", "NYSE", "OTC"])]

# Sample of 100 symbols
filtered_data = filtered_data.sample(n=100, random_state=42)

# Fetch, filter, and merge profile data 
filtered_and_merged_data = filter_and_merge_data(filtered_data, API_KEY)


Fetching and Filtering Data: 100%|██████████| 100/100 [01:02<00:00,  1.60it/s]


In [ ]:
# Print size and data type information of the filtered_data dataframe
print("Filtered Data Information:")
print("Number of rows:", len(filtered_and_merged_data))
print("Number of columns:", len(filtered_and_merged_data.columns))
print("\nData types:")
print(filtered_and_merged_data.dtypes)

------------------------------------------------------------------------

The next steps to perform the ranking is to download the financial data of the companies. Next section is to download the:

-   Income Statement

-   Balance Sheet

-   Cash Flow.

*Note: for the purpose of the magic formula the cash flow statement is not required. Nonetheless this is downloaded for additional evaluations of the financials.*

------------------------------------------------------------------------

In [9]:
# Extract the symbol list
symbol_list = filtered_and_merged_data['symbol']

# Download and aggreggate all financial statements
income_statement_df, balance_sheet_df, cash_flow_df = fetch_all_financial_statements(API_KEY, symbol_list)

# Merge filtered stock data with financial statement symbol list
income_statement_df = pd.merge(filtered_and_merged_data, income_statement_df, on="symbol")
balance_sheet_df = pd.merge(filtered_and_merged_data, balance_sheet_df, on="symbol")
cash_flow_df = pd.merge(filtered_and_merged_data, cash_flow_df, on="symbol")

---------------------------------------------------------------------------

In [17]:
# Convert column names to a dataframe or series
income_statement_columns = pd.DataFrame({'ColumnNames': income_statement_df.columns})
balance_sheet_columns = pd.DataFrame({'ColumnNames': balance_sheet_df.columns})
cash_flow_columns = pd.DataFrame({'ColumnNames': cash_flow_df.columns})

# Save the dataframes to CSV files
income_statement_columns.to_csv('columns_income_statement.csv', index=False)
balance_sheet_columns.to_csv('columns_balance_sheet.csv', index=False)
cash_flow_columns.to_csv('columns_cash_flow.csv', index=False)

